In [1]:
import pandas as pd
import pandas_gbq
import plotly.express as px
import pandas_gbq
import sys, requests
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option("styler.format.thousands", ",")

In [2]:
#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_team_boxscore.team_box_96_onwards`
"""
project_id = "perceptive-ivy-290216"
df_bq_reg_season = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
#USE FOR ALL TIME DATA
df_bq2=df_bq_reg_season.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year="2023-24"
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year].copy()

df_bq2=df_bq2.fillna(0)
df_bq2.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
0,21994,1610612737,ATL,Atlanta Hawks,0029400528,1995-01-26,ATL vs. CLE,L,240,25,65,0.385,3,18,0.167,15,23,0.652,8,23,31,13,9,4,14,17,68,-9,0,1994-95
1,21994,1610612737,ATL,Atlanta Hawks,0029401037,1995-04-14,ATL @ CLE,L,240,29,67,0.433,6,21,0.286,6,8,0.750,4,25,29,19,8,6,13,17,70,-13,0,1994-95
2,21994,1610612737,ATL,Atlanta Hawks,0029400381,1995-01-04,ATL @ NYK,L,240,27,68,0.397,5,21,0.238,21,25,0.840,8,25,33,16,9,5,18,21,80,-9,0,1994-95
3,21994,1610612737,ATL,Atlanta Hawks,0029400058,1994-11-12,ATL @ SAC,L,240,31,69,0.449,4,17,0.235,31,41,0.756,12,25,37,14,10,4,17,28,97,-8,0,1994-95
4,21994,1610612737,ATL,Atlanta Hawks,0029400902,1995-03-24,ATL @ CLE,L,240,29,70,0.414,6,25,0.240,10,11,0.909,8,34,42,19,6,7,18,19,74,-1,0,1994-95


In [4]:
df_bq2['FGMX']=df_bq2['FGA']-df_bq2['FGM']
df_bq2['FG3MX']=df_bq2['FG3A']-df_bq2['FG3M']
df_bq2['FG2M']=df_bq2['FGM']-df_bq2['FG3M']
df_bq2['FG2A']=df_bq2['FGA']-df_bq2['FG3A']
df_bq2['FG2_PCT']=df_bq2['FG2M']/df_bq2['FG2A']
df_bq2=df_bq2.round({'FG2_PCT': 3})

In [6]:
#Add Game Number for trended values plotting
df_bq2["Game Number"] = df_bq2.groupby("TEAM_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2["Game Number"]=df_bq2["Game Number"].astype(int)

In [8]:
#Get cumulative stat for each player
df_bq2['Total PTS'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PTS'].cumsum()
df_bq2['Total REB'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['REB'].cumsum()
df_bq2['Total DREB'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['DREB'].cumsum()
df_bq2['Total OREB'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['OREB'].cumsum()
df_bq2['Total AST'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['AST'].cumsum()
df_bq2['Total STL'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['STL'].cumsum()
df_bq2['Total BLK'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['BLK'].cumsum()
df_bq2['Total TOV'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['TOV'].cumsum()
df_bq2['Total PF'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PF'].cumsum()
df_bq2['Total FGM'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGM'].cumsum()
df_bq2['Total FGA'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGA'].cumsum()
df_bq2['Total FG3M'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3M'].cumsum()
df_bq2['Total FG3A'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3A'].cumsum()
df_bq2['Total FGMX'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGMX'].cumsum()
df_bq2['Total FG3MX'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3MX'].cumsum()
df_bq2['Total FTM'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTM'].cumsum()
df_bq2['Total FTA'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTA'].cumsum()
df_bq2['Total MIN'] = df_bq2.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['MIN'].cumsum()

In [11]:
#Get Totals For Each Category
df_totals=pd.DataFrame(df_bq2.groupby(by=['TEAM_NAME'])[['MIN', 'FGM',
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS','FGMX','FG3MX','FG2M','FG2A']].sum()).reset_index()
#Get Game Counts for Each Player
df_count=pd.DataFrame(df_bq2.groupby(by=['TEAM_NAME'])[["GAME_ID"]].count()).reset_index()
df_count.columns=["TEAM_NAME","GP"]
df_totals=df_totals.merge(df_count, left_on=["TEAM_NAME"],right_on=["TEAM_NAME"])
df_totals.head()

,TEAM_NAME,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FGMX,FG3MX,FG2M,FG2A,GP
0,Atlanta Hawks,755300,119866,261623,19902,56630,59823,78370,38678,95183,133861,70661,25377,15308,46723,66075,319457,-658,141757,36728,99964,204993,3125
1,Boston Celtics,756460,122370,262434,21123,59060,58721,75996,35315,97837,133152,74352,25439,15350,46486,68783,324584,6588,140064,37937,101247,203374,3130
2,Brooklyn Nets,233185,37806,82441,10412,29032,16932,22092,9752,32119,41871,22534,6822,4570,14009,19305,102956,-1460,44635,18620,27394,53409,964
3,Charlotte Bobcats,194885,28431,64541,4325,12500,14871,20199,8847,23698,32545,16913,5992,4102,11668,16907,76058,-3490,36110,8175,24106,52041,804
4,Charlotte Hornets,461310,73913,161873,13251,37033,36176,47323,21522,59197,80719,46691,15121,8846,27020,39070,197253,-3108,87960,23782,60662,124840,1909


In [12]:
#Add additional columns
df_totals['FG_PCT']=df_totals['FGM']/df_totals['FGA']
df_totals['FG3_PCT']=df_totals['FG3M']/df_totals['FG3A']
df_totals['FG2_PCT']=df_totals['FG2M']/df_totals['FG2A']
df_totals=df_totals.round({'FG_PCT': 3,'FG3_PCT': 3,'FG2_PCT': 3})

In [13]:
#Re-order the columns
df_totals=df_totals[['TEAM_NAME', 'GP', 'MIN', 'FGM', 'FGMX', 'FGA','FG_PCT', 'FG2M', 'FG2A','FG2_PCT','FG3M', 
          'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
# df_totals["MIN"]=df_totals["MIN"].apply(lambda x : "{:,}".format(x))
df_totals.head()

,TEAM_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,Atlanta Hawks,3125,755300,119866,141757,261623,0.458,99964,204993,0.488,19902,36728,56630,0.351,59823,78370,38678,95183,133861,70661,25377,15308,46723,66075,319457,-658
1,Boston Celtics,3130,756460,122370,140064,262434,0.466,101247,203374,0.498,21123,37937,59060,0.358,58721,75996,35315,97837,133152,74352,25439,15350,46486,68783,324584,6588
2,Brooklyn Nets,964,233185,37806,44635,82441,0.459,27394,53409,0.513,10412,18620,29032,0.359,16932,22092,9752,32119,41871,22534,6822,4570,14009,19305,102956,-1460
3,Charlotte Bobcats,804,194885,28431,36110,64541,0.441,24106,52041,0.463,4325,8175,12500,0.346,14871,20199,8847,23698,32545,16913,5992,4102,11668,16907,76058,-3490
4,Charlotte Hornets,1909,461310,73913,87960,161873,0.457,60662,124840,0.486,13251,23782,37033,0.358,36176,47323,21522,59197,80719,46691,15121,8846,27020,39070,197253,-3108


In [14]:
sort_category="PTS"

In [15]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals['RK_Category']=df_totals[sort_category].rank(method="dense", ascending=False)
df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
top=df_totals.sort_values(by=sort_category, ascending=False).head(10)
top

,TEAM_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,RK_Category
28,Phoenix Suns,3131,756265,126482,140843,267325,0.473,105595,209482,0.504,20887,36956,57843,0.361,60481,78805,36354,98075,134429,77253,25007,15637,47434,68128,334332,4708,1
10,Golden State Warriors,3123,754495,125690,145570,271260,0.463,103255,209647,0.493,22435,39178,61613,0.364,60109,79419,39149,97140,136289,76094,26369,16216,48735,70302,333924,-1437,2
8,Denver Nuggets,3131,756266,124541,146425,270966,0.46,105124,215309,0.488,19417,36240,55657,0.349,61576,81380,39512,97589,137101,75246,25730,17186,47248,70367,330075,-956,3
15,Los Angeles Lakers,3129,756660,124169,140948,265117,0.468,104077,207327,0.502,20092,37698,57790,0.348,61641,82765,37749,98882,136631,75187,24787,16667,45611,65518,330071,6605,4
29,Portland Trail Blazers,3132,757405,121934,142958,264892,0.46,101343,207297,0.489,20591,37004,57595,0.358,60591,79747,39615,96596,136211,71247,24117,15305,46491,68027,325050,4138,5
30,Sacramento Kings,3130,756400,122635,143859,266494,0.46,102931,211492,0.487,19704,35298,55002,0.358,60046,79491,37203,95984,133187,73030,24638,14449,47678,68364,325020,-6967,6
11,Houston Rockets,3130,756550,120056,141652,261708,0.459,95566,191937,0.498,24490,45281,69771,0.351,60018,79668,37059,98883,135942,71237,25093,16215,48391,65882,324620,4560,7
1,Boston Celtics,3130,756460,122370,140064,262434,0.466,101247,203374,0.498,21123,37937,59060,0.358,58721,75996,35315,97837,133152,74352,25439,15350,46486,68783,324584,6588,8
7,Dallas Mavericks,3133,757220,121396,143751,265147,0.458,98984,202409,0.489,22412,40326,62738,0.357,58650,75697,36772,96978,133750,69493,23225,14988,43046,66105,323854,1271,9
18,Milwaukee Bucks,3131,756615,121825,141363,263188,0.463,101395,206291,0.492,20430,36467,56897,0.359,58708,77299,36920,94862,131782,72773,25680,14458,46170,69170,322788,243,10


In [16]:
#Get Player Names of the Top Category we have selected
names=top.TEAM_NAME.to_list()

In [17]:
#Get the DF with Player Names from the top category
df_top=df_bq2[df_bq2['TEAM_NAME'].isin(names)]

In [18]:
#Get Trended Data Point 

In [19]:
fig = px.scatter(
  df_top, 
  x="Game Number", 
  y="Total {}".format(sort_category),
  color='TEAM_NAME',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
title="<b>Regular Season All Time Trended Player {} vs. Game Number</b>".format(sort_category),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
       'Game Number'], 
  height=800, 
  width=1200,
  )

fig.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig.update_traces(marker={'size': 3})

fig

In [ ]:
fig.write_html("Regular Season All Time Trended Team {} vs. Game Number.html".format(sort_category),full_html=False, include_plotlyjs='cdn')
# fig.write_html("Trended Team {} vs. Game Number For The {} Season.html".format(sort_category,current_year),full_html=False, include_plotlyjs='cdn')

In [230]:
#POLAR CHARTS BY TOTALS

In [54]:
polar=px.bar_polar(
top, 
r=sort_category, 
theta="TEAM_NAME", 
color="TEAM_NAME",
template='ggplot2',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
 title="Regular Season All Time Leaders In Team Total {}".format(sort_category),
 hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
       'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'RK_Category'],
 height=800, width=1200,
                  )
polar.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

In [ ]:
polar.write_html("Regular Season All Time Leaders Team In Total {}.html".format(sort_category),full_html=False, include_plotlyjs='cdn')
# polar.write_html("Team Leaders In Total {} For The {} Season.html".format(sort_category, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [55]:
cat1="GP"
cat2="AST"

fig_scatter = px.scatter(
  df_totals, 
  x=cat1, 
  y=cat2,
  color='TEAM_NAME',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},  title="<b>Regular Season All Time Team {} vs. {}</b>".format(cat1,cat2),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'], 
  height=800, 
  width=1200,
  )

fig_scatter.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter.update_traces(marker={'size': 10})

fig_scatter

In [ ]:
fig_scatter.write_html("Regular Season All Time Team {} vs. {}.html".format(cat1,cat2),full_html=False, include_plotlyjs='cdn')
# fig_scatter.write_html("Team {} vs. {} For The {} Season.html".format(cat1,cat2, current_year),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [22]:
sort_categories=['PTS', 'REB','DREB', 'OREB', 'AST', 'STL', 'BLK','TOV', 'PF', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FGMX', 'FG3MX', 'FTM',  'FTA','MIN']

In [ ]:
#TRENDED

In [56]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.TEAM_NAME.to_list()
    df_top=df_bq2[df_bq2['TEAM_NAME'].isin(names)]

    fig_multi = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='TEAM_NAME',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
    title="<b>Regular Season All Time Trended Team {} vs. Game Number</b>".format(i),
    template="plotly_white",
    hover_data=['PLAYER_NAME','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi.update_traces(marker={'size': 3})
    fig_multi.write_html("Regular Season All Time Trended Team {} vs. Game Number.html".format(i),full_html=False, include_plotlyjs='cdn')
    # fig_multi.write_html("Trended Team {} vs. Game Number For The {} Season.html".format(i, current_year),full_html=False, include_plotlyjs='cdn')    

AttributeError: 'DataFrame' object has no attribute 'PLAYER_NAME'

In [ ]:
#POLAR

In [ ]:
for i in sort_categories:
    df_totals['RK_Category']=df_totals[i].rank(method="dense", ascending=False)
    df_totals['RK_Category']=df_totals['RK_Category'].astype(int)
    top=df_totals.sort_values(by=i, ascending=False).head(10)
    names=top.PLAYER_NAME.to_list()
    df_top=df_bq2[df_bq2['TEAM_NAME'].isin(names)]
    
    polar_multi=px.bar_polar(
    top, 
    r=sort_category, 
    theta="TEAM_NAME", 
    color="TEAM_NAME",
    template='ggplot2',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
    title="Regular Season All Time Leaders In Total {}".format(i),
    hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
        'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
        'RK_Category'],
    height=800, width=1200,
                    )
    polar_multi.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    polar_multi.write_html("Regular Season All Time Leaders In Total Team {}.html".format(i),full_html=False, include_plotlyjs='cdn')
    # polar_multi.write_html("Leaders In Total Team {} For The {} Season.html".format(i, current_year),full_html=False, include_plotlyjs='cdn')    

In [ ]:
# PLAYOFFS

In [25]:
#Change the date to the previous day
query = """
SELECT * FROM `perceptive-ivy-290216.nba_api_team_boxscore.playoffs_team_box_96_onwards`
"""
project_id = "perceptive-ivy-290216"
df_bq_playoffs = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [26]:
#USE FOR ALL TIME DATA
df_bq2_playoffs=df_bq_playoffs.copy()

#USE FOR SINGLE SEASON PLOTS
# current_year_playoffs='2023-24'
# df_bq2=df_bq_reg_season[df_bq_reg_season["Season"]==current_year_playoffs].copy()

df_bq2_playoffs=df_bq2_playoffs.fillna(0)
df_bq2_playoffs.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE,Season
0,41985,1610612762,UTH,Utah Jazz,0048500003,1986-04-18,UTH @ DAL,L,240,41,84,0.488,0,1,0.0,11,14,0.786,13,23,36,31,7,8,13,23,93,-8,0,1985-86
1,41985,1610612762,UTH,Utah Jazz,0048500174,1986-04-25,UTH vs. DAL,L,240,47,90,0.522,3,5,0.6,16,29,0.552,17,26,43,27,7,6,10,23,113,-4,0,1985-86
2,41985,1610612762,UTH,Utah Jazz,0048500172,1986-04-20,UTH @ DAL,L,240,44,96,0.458,0,2,0.0,18,24,0.750,24,24,48,27,2,9,14,32,106,-7,0,1985-86
3,41986,1610612762,UTH,Utah Jazz,0048600028,1987-05-01,UTH @ GOS,L,240,37,74,0.500,3,6,0.5,17,24,0.708,14,23,37,19,9,5,25,24,94,-4,0,1986-87
4,41986,1610612762,UTH,Utah Jazz,0048600033,1987-05-03,UTH vs. GOS,L,240,40,85,0.471,5,10,0.5,28,41,0.683,19,22,41,23,8,11,13,26,113,-5,0,1986-87


In [27]:
df_bq2_playoffs['FGMX']=df_bq2_playoffs['FGA']-df_bq2_playoffs['FGM']
df_bq2_playoffs['FG3MX']=df_bq2_playoffs['FG3A']-df_bq2_playoffs['FG3M']
df_bq2_playoffs['FG2M']=df_bq2_playoffs['FGM']-df_bq2_playoffs['FG3M']
df_bq2_playoffs['FG2A']=df_bq2_playoffs['FGA']-df_bq2_playoffs['FG3A']
df_bq2_playoffs['FG2_PCT']=df_bq2_playoffs['FG2M']/df_bq2_playoffs['FG2A']
df_bq2_playoffs=df_bq2_playoffs.round({'FG2_PCT': 3})

In [29]:
#Add Game Number for trended values plotting
df_bq2_playoffs["Game Number"] = df_bq2_playoffs.groupby("TEAM_NAME")["GAME_DATE"].rank(method="dense", ascending=True)
df_bq2_playoffs["Game Number"]=df_bq2_playoffs["Game Number"].astype(int)

In [30]:
#Get cumulative stat for each player
df_bq2_playoffs['Total PTS'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PTS'].cumsum()
df_bq2_playoffs['Total REB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['REB'].cumsum()
df_bq2_playoffs['Total DREB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['DREB'].cumsum()
df_bq2_playoffs['Total OREB'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['OREB'].cumsum()
df_bq2_playoffs['Total AST'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['AST'].cumsum()
df_bq2_playoffs['Total STL'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['STL'].cumsum()
df_bq2_playoffs['Total BLK'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['BLK'].cumsum()
df_bq2_playoffs['Total TOV'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['TOV'].cumsum()
df_bq2_playoffs['Total PF'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['PF'].cumsum()
df_bq2_playoffs['Total FGM'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGM'].cumsum()
df_bq2_playoffs['Total FGA'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGA'].cumsum()
df_bq2_playoffs['Total FG3M'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3M'].cumsum()
df_bq2_playoffs['Total FG3A'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3A'].cumsum()
df_bq2_playoffs['Total FGMX'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FGMX'].cumsum()
df_bq2_playoffs['Total FG3MX'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FG3MX'].cumsum()
df_bq2_playoffs['Total FTM'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTM'].cumsum()
df_bq2_playoffs['Total FTA'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['FTA'].cumsum()
df_bq2_playoffs['Total MIN'] = df_bq2_playoffs.sort_values("GAME_DATE").groupby(['TEAM_NAME'])['MIN'].cumsum()

In [31]:
#Get Totals For Each Category
df_totals_playoffs=pd.DataFrame(df_bq2_playoffs.groupby(by=['TEAM_NAME'])[['MIN', 'FGM',
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS','PLUS_MINUS','FGMX','FG3MX','FG2M','FG2A']].sum()).reset_index()
#Get Game Counts for Each Player
df_count_playoffs=pd.DataFrame(df_bq2_playoffs.groupby(by=['TEAM_NAME'])[["GAME_ID"]].count()).reset_index()
df_count_playoffs.columns=["TEAM_NAME","GP"]
df_totals_playoffs=df_totals_playoffs.merge(df_count_playoffs, left_on=["TEAM_NAME"],right_on=["TEAM_NAME"])
df_totals_playoffs.head()

,TEAM_NAME,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,FGMX,FG3MX,FG2M,FG2A,GP
0,Anderson Packers,0,198,0,0,0,208,54,0,0,0,0,0,0,0,170,604,6,-198,0,198,0,8
1,Atlanta Hawks,66085,9989,17743,1338,4049,5333,7083,2394,6145,8929,4487,1465,1007,2845,6287,26649,-970,7754,2711,8651,13694,274
2,Baltimore Bullets,12795,2569,422,0,0,1652,1849,0,0,191,78,0,0,0,1383,6798,-142,-2147,0,2569,422,69
3,Boston Celtics,150705,29182,40249,2889,8193,16382,18835,4568,13234,20685,10726,3026,2100,5825,16643,77635,1785,11067,5304,26293,32056,738
4,Brooklyn Nets,13085,2011,4463,558,1580,976,1263,508,1674,2182,1137,368,224,692,1135,5556,-159,2452,1022,1453,2883,54


In [32]:
#Add additional columns
df_totals_playoffs['FG_PCT']=df_totals_playoffs['FGM']/df_totals_playoffs['FGA']
df_totals_playoffs['FG3_PCT']=df_totals_playoffs['FG3M']/df_totals_playoffs['FG3A']
df_totals_playoffs['FG2_PCT']=df_totals_playoffs['FG2M']/df_totals_playoffs['FG2A']
df_totals_playoffs=df_totals_playoffs.round({'FG_PCT': 3,'FG3_PCT': 3,'FG2_PCT': 3})

In [33]:
#Re-order the columns
df_totals_playoffs=df_totals_playoffs[['TEAM_NAME', 'GP', 'MIN', 'FGM', 'FGMX', 'FGA','FG_PCT', 'FG2M', 'FG2A','FG2_PCT','FG3M', 
          'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA','OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
# df_totals_playoffs["MIN"]=df_totals_playoffs["MIN"].apply(lambda x : "{:,}".format(x))
df_totals_playoffs.head()

,TEAM_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS
0,Anderson Packers,8,0,198,-198,0,inf,198,0,inf,0,0,0,NaN,208,54,0,0,0,0,0,0,0,170,604,6
1,Atlanta Hawks,274,66085,9989,7754,17743,0.563,8651,13694,0.632,1338,2711,4049,0.33,5333,7083,2394,6145,8929,4487,1465,1007,2845,6287,26649,-970
2,Baltimore Bullets,69,12795,2569,-2147,422,6.088,2569,422,6.088,0,0,0,NaN,1652,1849,0,0,191,78,0,0,0,1383,6798,-142
3,Boston Celtics,738,150705,29182,11067,40249,0.725,26293,32056,0.82,2889,5304,8193,0.353,16382,18835,4568,13234,20685,10726,3026,2100,5825,16643,77635,1785
4,Brooklyn Nets,54,13085,2011,2452,4463,0.451,1453,2883,0.504,558,1022,1580,0.353,976,1263,508,1674,2182,1137,368,224,692,1135,5556,-159


In [34]:
sort_category_playoffs="PTS"

In [35]:
#Sort by a category and then rank it. This will help get the Top Players in a category
df_totals_playoffs['RK_Category']=df_totals_playoffs[sort_category_playoffs].rank(method="dense", ascending=False)
df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
top_playoffs=df_totals_playoffs.sort_values(by=sort_category_playoffs, ascending=False).head(10)
top_playoffs

,TEAM_NAME,GP,MIN,FGM,FGMX,FGA,FG_PCT,FG2M,FG2A,FG2_PCT,FG3M,FG3MX,FG3A,FG3_PCT,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,RK_Category
3,Boston Celtics,738,150705,29182,11067,40249,0.725,26293,32056,0.82,2889,5304,8193,0.353,16382,18835,4568,13234,20685,10726,3026,2100,5825,16643,77635,1785,1
26,Los Angeles Lakers,678,154955,27299,14068,41367,0.66,24959,34524,0.723,2340,4503,6843,0.342,14953,19515,5431,13551,21577,11535,3334,2525,6263,14921,71892,1536,2
38,Philadelphia 76ers,391,93190,15320,6196,21516,0.712,14109,17949,0.786,1211,2356,3567,0.34,8818,11872,3028,7252,11135,5790,1973,1420,3720,8796,40669,60,3
44,San Antonio Spurs,403,97395,15125,13872,28997,0.522,12975,23169,0.56,2150,3678,5828,0.369,7810,10606,3900,11107,15301,7678,2516,1901,4900,8941,40210,726,4
35,New York Knicks,404,81555,14516,6819,21335,0.68,13350,17968,0.743,1166,2201,3367,0.346,8291,9488,2578,6123,10503,5349,1590,918,3135,10109,38509,-431,5
40,Phoenix Suns,324,78410,13001,9528,22529,0.577,11353,18097,0.627,1648,2784,4432,0.372,6690,8662,2758,7508,11154,6152,1735,1241,3203,7556,34340,-55,6
9,Chicago Bulls,349,84535,12933,10040,22973,0.563,11566,18959,0.61,1367,2647,4014,0.341,6846,9074,3689,8214,11903,6282,2233,1366,3828,8222,34079,363,7
29,Milwaukee Bucks,312,75530,12516,7658,20174,0.62,11039,15923,0.693,1477,2774,4251,0.347,6086,8109,2415,6666,10323,5390,1554,1047,2963,7434,32595,208,8
19,Houston Rockets,316,76340,11954,10217,22171,0.539,9906,16261,0.609,2048,3862,5910,0.347,6240,8377,3131,8345,11476,5848,2085,1456,3847,7182,32196,-127,9
16,Detroit Pistons,326,71325,11929,9836,21765,0.548,10860,18577,0.585,1069,2119,3188,0.335,6780,8656,3282,7574,11079,5465,1797,1261,3420,7528,31707,252,10


In [37]:
#Get Player Names of the Top Category we have selected
names_playoffs=top_playoffs.TEAM_NAME.to_list()

In [38]:
#Get the DF with Player Names from the top category
df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['TEAM_NAME'].isin(names_playoffs)]

In [39]:
#Get Trended Data Point 

In [42]:
fig_playoffs = px.scatter(
  df_top_playoffs, 
  x="Game Number", 
  y="Total {}".format(sort_category_playoffs),
  color='TEAM_NAME',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
  title="<b>Playoffs All Time Trended Team {} vs. Game Number</b>".format(sort_category),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
       'Game Number'], 
  height=800, 
  width=1200,
  )

fig_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_playoffs.update_traces(marker={'size': 3})

fig_playoffs

In [ ]:
fig_playoffs.write_html("Playoffs All Time Trended Team {} vs. Game Number.html".format(sort_category_playoffs),full_html=False, include_plotlyjs='cdn')
# fig_playoffs.write_html("Trended Team {} vs. Game Number For The {} Playoffs.html".format(sort_category_playoffs,current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#POLAR CHARTS BY TOTALS

In [53]:
polar_playoffs=px.bar_polar(
top_playoffs, 
r=sort_category, 
theta="TEAM_NAME", 
color="TEAM_NAME",
template='ggplot2',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
 title="Playoffs All Time Leaders In Total {}".format(sort_category),
 hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
       'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
       'RK_Category'],
 height=800, width=1200,
                  )
polar_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)

In [ ]:
polar_playoffs.write_html("Playoffs All Time Leaders In Total Team {}.html".format(sort_category_playoffs),full_html=False, include_plotlyjs='cdn')
# polar_playoffs.write_html("Leaders In Total Team {} For The {} Playoffs.html".format(sort_category_playoffs, current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#2 variable scatter plot

In [45]:
cat1_playoffs="GP"
cat2_playoffs="AST"

fig_scatter_playoffs = px.scatter(
  df_totals_playoffs, 
  x=cat1, 
  y=cat2,
  color='TEAM_NAME',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
  title="<b>Playoffs All Time Player {} vs. {}</b>".format(cat1,cat2),
  template="plotly_white",
  hover_data=['TEAM_NAME', 'GP', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
       'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS'], 
  height=800, 
  width=1200,
  )

fig_scatter_playoffs.update_layout(
    title_x=0.5,
    hoverlabel=dict(
        bgcolor="white",
        font_size=16,
        font_family="PT Sans Narrow"
    ),
    yaxis = dict(tickfont = dict(size=20)),
    xaxis = dict(tickfont = dict(size=20)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
)
fig_scatter_playoffs.update_traces(marker={'size': 10})

fig_scatter_playoffs

In [ ]:
fig_scatter_playoffs.write_html("Playoffs All Time Team {} vs. {}.html".format(cat1_playoffs,cat2_playoffs),full_html=False, include_plotlyjs='cdn')
# fig_scatter_playoffs.write_html("Team {} vs. {} For The {} Playoffs.html".format(cat1_playoffs,cat2_playoffs, current_year_playoffs),full_html=False, include_plotlyjs='cdn')

In [ ]:
#MULTI METRIC PLOTS TOGETHER

In [47]:
sort_categories_playoffs=['PTS', 'REB','DREB', 'OREB', 'AST', 'STL', 'BLK','TOV', 'PF', 'FGM', 'FGA', 'FG3M',
       'FG3A', 'FGMX', 'FG3MX', 'FTM',  'FTA','MIN']

In [48]:
#TRENDED

In [49]:
for i in sort_categories_playoffs:
    df_totals_playoffs['RK_Category']=df_totals_playoffs[i].rank(method="dense", ascending=False)
    df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
    top_playoffs=df_totals_playoffs.sort_values(by=i, ascending=False).head(10)
    names_playoffs=top_playoffs.TEAM_NAME.to_list()
    df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['TEAM_NAME'].isin(names)]

    fig_multi_playoffs = px.scatter(
    df_top, 
    x="Game Number", 
    y="Total {}".format(i),
    color='TEAM_NAME',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
    title="<b>All Time Trended Team {} vs. Game Number</b>".format(i),
    template="plotly_white",
    hover_data=['TEAM_NAME','Total PTS', 'Total REB',
       'Total DREB', 'Total OREB', 'Total AST', 'Total STL', 'Total BLK',
       'Total TOV', 'Total PF', 'Total FGM', 'Total FGA', 'Total FG3M',
       'Total FG3A', 'Total FGMX', 'Total FG3MX', 'Total FTM', 'Total FTA',
       'Total MIN'
        #          'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        # 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS', 'Season',
        # 'Game Number'
        ], 
    height=800, 
    width=1200,
    )

    fig_multi_playoffs.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    fig_multi_playoffs.update_traces(marker={'size': 3})
    fig_multi_playoffs.write_html("Playoffs All Time Trended Team {} vs. Game Number.html".format(i),full_html=False, include_plotlyjs='cdn')
    # fig_multi_playoffs.write_html("Trended Team {} vs. Game Number For The {} Playoffs.html".format(i, current_year_playoffs),full_html=False, include_plotlyjs='cdn')    

In [ ]:
#POLAR

In [52]:
for i in sort_categories_playoffs:
    df_totals_playoffs['RK_Category']=df_totals_playoffs[i].rank(method="dense", ascending=False)
    df_totals_playoffs['RK_Category']=df_totals_playoffs['RK_Category'].astype(int)
    top_playoffs=df_totals_playoffs.sort_values(by=i, ascending=False).head(10)
    names_playoffs=top_playoffs.TEAM_NAME.to_list()
    df_top_playoffs=df_bq2_playoffs[df_bq2_playoffs['TEAM_NAME'].isin(names)]
    
    polar_multi_playoffs=px.bar_polar(
    top_playoffs, 
    r=i, 
    theta="TEAM_NAME", 
    color="TEAM_NAME",
    template='ggplot2',
color_discrete_map={"Atlanta Hawks":"#C8102E",
"Charlotte Bobcats":"#E35205",
"Brooklyn Nets":"#707372",
"Boston Celtics":"#007A33",
"Seattle SuperSonics":"#00653A",
"Charlotte Hornets":"#00778B",
"Buffalo Braves":"#418FDE",
"Chicago Bulls":"#BA0C2F",
"Cleveland Cavaliers":"#6F263D",
"Dallas Mavericks":"#0050B5",
"Denver Nuggets":"#1d428a",
"Detroit Pistons":"#1D4289",
"Golden State Warriors":"#1D4289",
"Houston Rockets":"#BA0C2F",
"Indiana Pacers":"#041E42",
"Los Angeles Clippers":"#0C2340",
"Los Angeles Lakers":"#6a00ed",
"Memphis Grizzlies":"#0C2340",
"Miami Heat":"#862633",
"Milwaukee Bucks":"#2C5234",
"Minnesota Timberwolves":"#236192",
"New Orleans Pelicans":"#85714d",
"New York Knicks":"#FF8200",
"Oklahoma City Thunder":"#0072CE",
"Orlando Magic":"#0057B7",
"Philadelphia 76ers":"#1D4289",
"Phoenix Suns":"#e56020",
"Portland Trail Blazers":"#C8102E",
"Sacramento Kings":"#965ddd",
"San Antonio Spurs":"#9EA2A2",
"Toronto Raptors":"#C8102E",
"Utah Jazz":"#0C2340",
"Washingotn Wizards":"#C8102E",
"Rochester Royals":"#003DA5",
"Washington Bullets":"#C8102E"},
    title="All Time Leaders In Total {}".format(i),
    hover_data=['TEAM_NAME', 'MIN', 'FGM', 'FGMX', 'FGA', 'FG_PCT', 'FG2M', 'FG2A',
        'FG2_PCT', 'FG3M', 'FG3MX', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'OREB',
        'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS',
        'RK_Category'],
    height=800, width=1200,
                    )
    polar_multi_playoffs.update_layout(
        title_x=0.5,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="PT Sans Narrow"
        ),
        yaxis = dict(tickfont = dict(size=20)),
        xaxis = dict(tickfont = dict(size=20)),
        font=dict(
            family="PT Sans Narrow",
            size=14,
            color="Black"
        ),
        title_font_family="PT Sans Narrow"
    )
    polar_multi_playoffs.write_html("Playoffs All Time Leaders In Total Team {}.html".format(i),full_html=False, include_plotlyjs='cdn')
    # polar_multi_playoffs.write_html("Leaders In Total Team {} For The {} Playoffs.html".format(i, current_year_playoffs),full_html=False, include_plotlyjs='cdn')    